In [1]:
import requests
from bs4 import BeautifulSoup
import json

def get_vacancies_from_headhunter(query, num_pages=1):
    """
    Функция для сбора вакансий с сайта HeadHunter по запросу.
    :param query: Строка запроса (например, 'Python Developer')
    :param num_pages: Количество страниц для сбора вакансий
    :return: Список вакансий в формате JSON
    """
    base_url = 'https://hh.ru/search/vacancy'
    vacancies = []
    
    for page in range(num_pages):
        params = {'text': query, 'page': page}
        response = requests.get(base_url, params=params)
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for vacancy in soup.find_all('div', {'class': 'vacancy-serp-item'}):
            title = vacancy.find('a', {'class': 'bloko-link'}).text.strip()
            company = vacancy.find('div', {'class': 'vacancy-serp-item__meta-info'}).text.strip()
            link = vacancy.find('a', {'class': 'bloko-link'})['href']
            
            vacancies.append({
                'title': title,
                'company': company,
                'link': link
            })
    
    return vacancies

# Пример запроса
vacancies = get_vacancies_from_headhunter('Python Developer', num_pages=2)
print(json.dumps(vacancies, indent=4, ensure_ascii=False))


[]


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Данные
resume_texts = ["Experienced in Python and Machine Learning", "Expert in AWS and cloud technologies", "Skilled in data analysis and SQL"]
job_descriptions = ["Looking for a Python developer with ML expertise", "Hiring for an AWS specialist", "Seeking a SQL data analyst"]
labels = [1, 1, 1]  # Метки, где 1 означает соответствие, а 0 - несоответствие

# Преобразование текста в числовые векторы
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(resume_texts + job_descriptions).toarray()

# Размерность входного слоя
input_dim = X.shape[1]

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, labels * 2, test_size=0.2, random_state=42)

# Модель
model = Sequential([
    Dense(128, activation='relu', input_shape=(input_dim,)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size=2, validation_data=(np.array(X_test), np.array(y_test)))

# Пример использования модели для предсказания подходящих вакансий
def predict_vacancy_match(resume, vacancies):
    resume_vector = vectorizer.transform([resume]).toarray()
    results = []
    
    for vacancy in vacancies:
        vacancy_vector = vectorizer.transform([vacancy['title'] + ' ' + vacancy['company']]).toarray()
        match_score = model.predict(np.array([np.concatenate((resume_vector, vacancy_vector))]))
        results.append({
            'title': vacancy['title'],
            'company': vacancy['company'],
            'link': vacancy['link'],
            'match_score': match_score[0][0]
        })
    
    return results

# Пример предсказания для резюме и вакансий с сайта HeadHunter
resume = "Experienced in Python and data analysis"
vacancy_matches = predict_vacancy_match(resume, vacancies)
print(json.dumps(vacancy_matches, indent=4, ensure_ascii=False))


Epoch 1/10
2/2 [==============================] - 1s 212ms/step - loss: 0.7760 - accuracy: 0.0000e+00 - val_loss: 0.7435 - val_accuracy: 0.0000e+00
Epoch 2/10
2/2 [==============================] - 0s 31ms/step - loss: 0.7324 - accuracy: 0.2500 - val_loss: 0.7183 - val_accuracy: 0.5000
Epoch 3/10
2/2 [==============================] - 0s 32ms/step - loss: 0.6965 - accuracy: 0.2500 - val_loss: 0.6945 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 24ms/step - loss: 0.6647 - accuracy: 0.5000 - val_loss: 0.6726 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 0s 27ms/step - loss: 0.6355 - accuracy: 1.0000 - val_loss: 0.6515 - val_accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 0s 25ms/step - loss: 0.6093 - accuracy: 1.0000 - val_loss: 0.6290 - val_accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 33ms/step - loss: 0.5853 - accuracy: 1.0000 - val_loss: 0.6074 - val_accuracy: 1.0000
Epoch 8/10
2/2 [==

In [3]:
import json

def save_vacancy_matches_to_json(vacancy_matches, filename='vacancy_matches.json'):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(vacancy_matches, f, ensure_ascii=False, indent=4)

# Сохраняем результат в JSON файл
save_vacancy_matches_to_json(vacancy_matches)


In [5]:
import streamlit as st

ModuleNotFoundError: No module named 'streamlit'

In [4]:


# Интерфейс
st.title("Интеллектуальный подбор вакансий")

resume_input = st.text_area("Введите текст резюме")

if st.button("Поиск вакансий"):
    if resume_input:
        # Получаем вакансии с сайта HeadHunter
        vacancies = get_vacancies_from_headhunter('Python Developer', num_pages=2)
        
        # Получаем подходящие вакансии
        vacancy_matches = predict_vacancy_match(resume_input, vacancies)
        
        # Отображаем результаты
        st.write("Подходящие вакансии:")
        for match in vacancy_matches:
            st.write(f"**{match['title']}** ({match['company']})")
            st.write(f"Ссылка: {match['link']}")
            st.write(f"Оценка соответствия: {match['match_score']:.2f}")
    else:
        st.error("Введите текст резюме!")


ModuleNotFoundError: No module named 'streamlit'